<a href="https://colab.research.google.com/github/cannin/gsoc_2023_pytorch_pathway_commons/blob/main/Modelling_with_Breast_Cancer_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch-geometric

# Importing Data and Libraries

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

if device.type != 'cuda':
    raise SystemError('GPU device not found')

Using device: cuda


In [ ]:
Brca_patients = pd.read_csv('/content/drive/MyDrive/gsoc data/Brca_tcga_2018/Brca_patients.csv')

In [ ]:
Brca_patients = Brca_patients.set_index('Sample Identifier')
Brca_patients.index.name = None
Brca_patients.head(2)

,Overall Survival (Months),UBE2Q2P2,HMGB1P1,RNU12-2P,SSX9P,EZHIP,EFCAB8,SRP14P1,TRIM75P,SPATA31B1P,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
TCGA-3C-AAAU-01,133.050597,12.9316,52.1503,0.0000,0.0000,1.7233,0.3447,4.1359,0.6893,0.0,...,415.824,931.957,129.5920,1007.780,1658.50,258.494,1208.370,3507.25,1894.93,1180.460
TCGA-3C-AALI-01,131.669790,17.3790,69.7553,1.0875,0.5438,144.1000,4.8940,1.6313,0.5438,0.0,...,1161.330,965.198,59.8151,448.613,1343.12,198.477,603.589,5504.62,1318.65,406.743


In [ ]:
#defining features and labels

y = np.array(Brca_patients['Overall Survival (Months)'], dtype=float)
X = Brca_patients.drop('Overall Survival (Months)', axis=1)

In [ ]:
path = "/content/drive/MyDrive/gsoc data/PathwayCommons12.reactome.hgnc.sif.gz"


#specify compression type because the file is compressed
df = pd.read_csv(path, sep="\t", compression="gzip", header=None, names=["Source", "InteractionType", "Target"])

In [ ]:
df = df.drop(columns='InteractionType')

In [ ]:
df.head()

,Source,Target
0,A1CF,APOBEC1
1,A1CF,APOBEC2
2,A1CF,APOBEC3A
3,A1CF,APOBEC3B
4,A1CF,APOBEC3C


In [ ]:
X.shape, y.shape

((1082, 20511), (1082,))

# Selecting Common Nodes From Both Data

In [ ]:
all_nodes = sorted(list(set(df['Source']) | set(df['Target'])))
len(all_nodes)

12324

In [ ]:
len(set(all_nodes) & set(X.columns))

9614

In [ ]:
used_nodes = sorted(list(set(X.columns) & set(all_nodes)))
len(used_nodes)

9614

In [ ]:
X = X.loc[:, used_nodes]
X.head(2)

,A1CF,A2M,A4GNT,AAAS,AACS,AADAC,AADAT,AAK1,AANAT,AARS,...,ZP2,ZP3,ZP4,ZRANB1,ZRSR2,ZSCAN10,ZW10,ZWILCH,ZWINT,ZZZ3
TCGA-3C-AAAU-01,0.0,5798.37,8.6165,649.337,1087.400,0.0000,5.5145,517.333,0.3447,4409.56,...,0.0000,411.791,0.0000,879.225,421.862,2.7573,430.824,415.824,931.957,1180.460
TCGA-3C-AALI-01,0.0,7571.98,0.5438,887.983,836.324,1.0875,24.4698,848.287,2.1751,4570.96,...,4.3502,439.222,0.5438,566.068,349.647,0.0000,340.402,1161.330,965.198,406.743


In [ ]:
# Extract the values from the 'Source' column of the DataFrame
source_values = df['Source'].values

# Create a boolean mask indicating whether each element in 'source_values' is present in 'used_nodes'
mask = np.isin(source_values, used_nodes)

df = df.loc[mask]

# Update 'source_values' with the values from the 'Target' column of the filtered DataFrame
source_values = df['Target'].values

# Create a new boolean mask based on the updated 'source_values' array
mask = np.isin(source_values, used_nodes)


df = df.loc[mask]

# Reset the index of the filtered DataFrame, dropping the old index
df = df.reset_index(drop=True)

In [ ]:
len(set(df['Source']) | set(df['Target']))

9288

In [ ]:

# Select the columns from DataFrame 'X' that correspond to unique values
columns_to_select = sorted(set(df['Source']).union(set(df['Target'])))
X = X[columns_to_select]
X.head(2)

,A1CF,A2M,A4GNT,AAAS,AACS,AADAT,AAK1,AANAT,AARS,AARS2,...,ZP2,ZP3,ZP4,ZRANB1,ZRSR2,ZSCAN10,ZW10,ZWILCH,ZWINT,ZZZ3
TCGA-3C-AAAU-01,0.0,5798.37,8.6165,649.337,1087.400,5.5145,517.333,0.3447,4409.56,689.897,...,0.0000,411.791,0.0000,879.225,421.862,2.7573,430.824,415.824,931.957,1180.460
TCGA-3C-AALI-01,0.0,7571.98,0.5438,887.983,836.324,24.4698,848.287,2.1751,4570.96,472.501,...,4.3502,439.222,0.5438,566.068,349.647,0.0000,340.402,1161.330,965.198,406.743


In [ ]:
X.shape

(1082, 9288)

# Creating Edge Index from Pathway Commons

In [ ]:
source_nodes = df['Source'].tolist()
target_nodes = df['Target'].tolist()


# Create a dictionary to map each unique node to a unique index
node_to_index = {node: index for index, node in enumerate(X.columns)}

# Map the source and target nodes to their corresponding indices
source_indices = [node_to_index[node] for node in source_nodes]
target_indices = [node_to_index[node] for node in target_nodes]

In [ ]:
# Convert the source and target indices to a PyTorch tensor
edge_index = torch.tensor([source_indices, target_indices], dtype=torch.long)

In [ ]:
edge_index

tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
        [ 451,  452,  453,  ..., 3323, 3340, 3341]])

In [ ]:
edge_index.shape

torch.Size([2, 271771])

# Splitting the Data into Train and Test Splits

In [ ]:
# Set the fixed index for splitting
split_index = int(0.8 * X.shape[0])

# Split the data based on the fixed index
X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]

In [ ]:
X_train.shape, X_test.shape

((865, 9288), (217, 9288))

In [ ]:
X_train = X_train.values
X_test = X_test.values

In [ ]:
X_train[0]

array([   0.    , 5798.37  ,    8.6165, ...,  415.824 ,  931.957 ,
       1180.46  ])

In [ ]:
X_test[1]

array([0.00000e+00, 2.28575e+04, 1.28340e+00, ..., 2.94419e+02,
       2.65659e+02, 8.94513e+02])

In [ ]:
y_train

array([1.33050597e+02, 1.31669790e+02, 4.84597429e+01, 4.76049577e+01,
       1.14409705e+01, 4.85583720e+01, 9.96153467e+00, 8.51497518e+00,
       1.43669658e+01, 4.34296610e+01, 4.80981030e+01, 1.42683368e+01,
       4.72433179e+01, 2.08764835e+01, 1.36765625e+01, 3.17914324e+01,
       7.95607719e+00, 3.93201170e+01, 2.80106519e+01, 1.91997896e+01,
       1.82134990e+01, 1.43143637e+02, 1.80162409e+01, 7.84100996e+01,
       1.21938390e+02, 7.38402867e+01, 8.72538383e+01, 6.31225959e+01,
       1.01982444e+02, 5.54295295e+01, 3.61311109e+01, 1.36732748e+02,
       9.91222014e+01, 2.47887694e+01, 1.14804221e+02, 9.24811783e+01,
       8.86017688e+01, 1.07933064e+02, 7.71936746e+01, 7.52539698e+01,
       5.19446362e+00, 9.89906960e+01, 1.07933064e+02, 5.68103363e+01,
       5.31281849e+01, 6.73307690e+01, 3.45530460e+01, 3.37640136e+01,
       6.15774074e+01, 2.52161620e+01, 1.01719433e+02, 1.34398527e+02,
       8.02840517e+01, 1.18453496e+02, 7.97580301e+01, 7.43991847e+01,
      

# Generating patient-specific graphs

In [ ]:
num_patients_train = X_train.shape[0]
num_patients_test = X_test.shape[0]

# Create patient-specific graphs for the training set
graphs_train = []
for i in range(num_patients_train):
    node_features = X_train[i]  # Node features for the i-th patient
    target = y_train[i]  # Target label for the i-th patient
    graph_train = (node_features, edge_index, target)
    graphs_train.append(graph_train)

# Create patient-specific graphs for the test set
graphs_test = []
for i in range(num_patients_test):
    node_features = X_test[i]  # Node features for the i-th patient
    target = y_test[i]  # Target label for the i-th patient
    graph_test = (node_features, edge_index, target)
    graphs_test.append(graph_test)

In [ ]:
# Check the number of patient-specific graphs
print(len(graphs_train))  # Should be 857
print(len(graphs_test))  # Should be 217

865
217


In [ ]:
# Access the patient-specific graph for a specific patient in the training set
patient_index = 0 # Index of the patient
node_features, edge_index, y = graphs_train[patient_index]
print(node_features)  # Node features for the specific patient
print(edge_index)  # Edge index for the specific patient
print(y)

[   0.     5798.37      8.6165 ...  415.824   931.957  1180.46  ]
tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
        [ 451,  452,  453,  ..., 3323, 3340, 3341]])
133.0505967


In [ ]:
# Access the patient-specific graph for a specific patient in the training set
patient_index = 15  # Index of the patient
node_features, edge_index, y = graphs_test[patient_index]
print(node_features)  # Node features for the specific patient
print(edge_index)  # Edge index for the specific patient
# print(y)

[0.00000e+00 2.87393e+04 9.46600e-01 ... 5.60854e+02 7.04274e+02
 5.87369e+02]
tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
        [ 451,  452,  453,  ..., 3323, 3340, 3341]])


# Converting List of Graphs to Data Objects

In [ ]:
# Convert graphs_train to a list of Data objects
data_train = [Data(x=torch.tensor(graph[0].reshape(len(graphs_train[0][0]), 1)), edge_index=graph[1], y=torch.tensor(graph[2])) for graph in graphs_train]

# Convert graphs_test to a list of Data objects
data_test = [Data(x=torch.tensor(graph[0].reshape(len(graphs_test[0][0]), 1)), edge_index=graph[1], y=torch.tensor(graph[2])) for graph in graphs_test]

In [ ]:
# data_test

In [ ]:
# Access the attributes of a specific data object in the training set
sample = data_train[0]  # Get the first data object
print(sample)  # Print the data object

# Access the node features, edge indices, and target label
node_features = sample.x
edge_index = sample.edge_index
target = sample.y

print(node_features)  # Print the node features
print(edge_index)  # Print the edge indices
print(target)  # Print the target label

Data(x=[9288, 1], edge_index=[2, 271771], y=133.0505967)
tensor([[   0.0000],
        [5798.3700],
        [   8.6165],
        ...,
        [ 415.8240],
        [ 931.9570],
        [1180.4600]], dtype=torch.float64)
tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
        [ 451,  452,  453,  ..., 3323, 3340, 3341]])
tensor(133.0506, dtype=torch.float64)


# Creating Train and Test Batches

In [ ]:
from torch_geometric.loader import DataLoader

In [ ]:
bs = 16
train_loader = DataLoader(data_train, batch_size=bs, shuffle=True)
test_loader = DataLoader(data_test, batch_size=bs, shuffle=False)

for step, data in enumerate(train_loader):
    data = data.to(device)  # Move the batch of data to the device

    print('Training Batches: ')
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Training Batches: 
Step 1:
Number of graphs in the current batch: 16
DataBatch(x=[148608, 1], edge_index=[2, 4348336], y=[16], batch=[148608], ptr=[17])

Training Batches: 
Step 2:
Number of graphs in the current batch: 16
DataBatch(x=[148608, 1], edge_index=[2, 4348336], y=[16], batch=[148608], ptr=[17])

Training Batches: 
Step 3:
Number of graphs in the current batch: 16
DataBatch(x=[148608, 1], edge_index=[2, 4348336], y=[16], batch=[148608], ptr=[17])

Training Batches: 
Step 4:
Number of graphs in the current batch: 16
DataBatch(x=[148608, 1], edge_index=[2, 4348336], y=[16], batch=[148608], ptr=[17])

Training Batches: 
Step 5:
Number of graphs in the current batch: 16
DataBatch(x=[148608, 1], edge_index=[2, 4348336], y=[16], batch=[148608], ptr=[17])

Training Batches: 
Step 6:
Number of graphs in the current batch: 16
DataBatch(x=[148608, 1], edge_index=[2, 4348336], y=[16], batch=[148608], ptr=[17])

Training Batches: 
Step 7:
Number of graphs in the current batch: 16
DataBat

In [ ]:
for step, data in enumerate(test_loader):
    data = data.to(device)
    print('Test Batches: ')
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Test Batches: 
Step 1:
Number of graphs in the current batch: 16
DataBatch(x=[148608, 1], edge_index=[2, 4348336], y=[16], batch=[148608], ptr=[17])

Test Batches: 
Step 2:
Number of graphs in the current batch: 16
DataBatch(x=[148608, 1], edge_index=[2, 4348336], y=[16], batch=[148608], ptr=[17])

Test Batches: 
Step 3:
Number of graphs in the current batch: 16
DataBatch(x=[148608, 1], edge_index=[2, 4348336], y=[16], batch=[148608], ptr=[17])

Test Batches: 
Step 4:
Number of graphs in the current batch: 16
DataBatch(x=[148608, 1], edge_index=[2, 4348336], y=[16], batch=[148608], ptr=[17])

Test Batches: 
Step 5:
Number of graphs in the current batch: 16
DataBatch(x=[148608, 1], edge_index=[2, 4348336], y=[16], batch=[148608], ptr=[17])

Test Batches: 
Step 6:
Number of graphs in the current batch: 16
DataBatch(x=[148608, 1], edge_index=[2, 4348336], y=[16], batch=[148608], ptr=[17])

Test Batches: 
Step 7:
Number of graphs in the current batch: 16
DataBatch(x=[148608, 1], edge_index

# Model Building and Evaluation

In [ ]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

GATConv

GraphNorm

Dropout for all layers

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, num_node_features):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.conv4 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 1)  # Regression output with 1 dimension

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()
        # x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        # F.dropout(x, p=0.2, training=self.training)
        x = self.conv3(x, edge_index)
        x = self.conv4(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final regression layer
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin(x)

        return x.squeeze()  # Remove the extra dimension

In [ ]:
model = GCN(hidden_channels=64, num_node_features=1)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

model = model.to(device)  # Move the model to CUDA device
criterion = criterion.to(device)  # Move the criterion to CUDA device

model.double()  # Convert the model's parameters to Double type

num_epochs = 101  # Specify the number of epochs

for epoch in range(num_epochs):
    model.train()  # Set the model to train mode
    total_loss = 0

    for step, data in enumerate(train_loader):
        data = data.to(device)  # Move the batch of data to CUDA device

        optimizer.zero_grad()

        out = model(data.x.double(), data.edge_index, data.batch)
        loss = criterion(out, data.y.view(-1).double())
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # average_loss = total_loss / (step + 1)
    average_loss = total_loss / len(train_loader)

    # Evaluate on the test set
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        test_loss = 0
        for data in test_loader:
            data = data.to(device)  # Move the batch of data to CUDA device

            out = model(data.x.double(), data.edge_index, data.batch)
            loss = criterion(out, data.y.view(-1).double())
            test_loss += loss.item()

        average_test_loss = test_loss / len(test_loader)
        print(f'Epoch: {epoch:03d}, Train loss: {average_loss:.4f}, Test Loss: {average_test_loss:.4f}')


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 000, Train loss: 8579.2108, Test Loss: 721.5855
Epoch: 001, Train loss: 4935.7021, Test Loss: 821.8742
Epoch: 002, Train loss: 3640.0271, Test Loss: 960.0009
Epoch: 003, Train loss: 3335.8440, Test Loss: 891.0839
Epoch: 004, Train loss: 2667.9678, Test Loss: 1235.8527
Epoch: 005, Train loss: 2722.5816, Test Loss: 774.1534
Epoch: 006, Train loss: 2520.1283, Test Loss: 766.3230
Epoch: 007, Train loss: 2216.6224, Test Loss: 722.5284
Epoch: 008, Train loss: 2094.2834, Test Loss: 997.0398
Epoch: 009, Train loss: 2016.0662, Test Loss: 1038.2014
Epoch: 010, Train loss: 2037.6068, Test Loss: 865.6619
Epoch: 011, Train loss: 1994.0958, Test Loss: 896.2507


KeyboardInterrupt: ignored

In [ ]:
is_cuda = next(model.parameters()).is_cuda

if is_cuda:
    print("The model is using CUDA (GPU) for computation.")
else:
    print("The model is not using CUDA (GPU) for computation.")

The model is not using CUDA (GPU) for computation.


In [ ]:
model.eval()  # Set the model to evaluation mode

predictions = []  # List to store the predicted outputs

with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        out = model(data.x.double(), data.edge_index, data.batch)
        predictions.append(out.detach().cpu().numpy())  # Convert the predictions to NumPy array

# Concatenate the predictions from multiple batches
predictions = np.concatenate(predictions)

# Print the predictions
# print(predictions)

In [ ]:
from sklearn.metrics import r2_score

# Convert the test data batches to a list of Data objects
test_data_batches = [
    Data(x=batch.x, edge_index=batch.edge_index, y=batch.y) for batch in test_loader
]

# Convert the predictions to PyTorch tensors
predictions = torch.tensor(predictions)

# Convert the ground truth labels of the test data to a PyTorch tensor
y_true = torch.cat([batch.y for batch in test_data_batches])

# Calculate the mean squared error (MSE) loss using PyTorch's function
mse_loss = torch.nn.functional.mse_loss(predictions.view(-1), y_true.view(-1))

print(f"Mean Squared Error (MSE) Loss: {mse_loss:.4f}")

Mean Squared Error (MSE) Loss: 705.2236
